In [53]:
#unicodes
'∃'
'∀'
'¬'
'∧'
'∨'
'⇒'
'⇔'
import re

In [57]:
def getAttributes(string): #to get the arguments inside
    expr = '\([^)]+\)'
    matches = re.findall(expr, string)
    return [m for m in str(matches) if m.isalpha()]

def getPredicates(string):  #to get the predicate name
    expr = '[a-z¬]+\([A-Za-z,]+\)'
    return re.findall(expr, string)

def deMorConv(sentence):
    string = ''.join(list(sentence).copy())
    string = string.replace('¬¬','')
    flag = '[' in string
    string = string.replace('¬[','')
    string = string.strip(']')
    for predicate in getPredicates(string):
        string = string.replace(predicate, f'¬{predicate}')
    s = list(string)
    for i, c in enumerate(string):
        if c == '∨':
            s[i] = '∧'
        elif c == '∧':
            s[i] = '∨'
    string = ''.join(s)    
    string = string.replace('¬¬','')
    return f'[{string}]' if flag else string

def Skolemization(sentence):
    #to generate skolem constants or funcions
    SKOLEM_CONSTANTS = [f'{chr(c)}' for c in range(ord('A'), ord('Z')+1)]
    sentence = ''.join(list(sentence).copy())
    matches = re.findall('[∀∃].', sentence)
    for match in matches[::-1]:
        sentence = sentence.replace(match, '')
        sentences = re.findall('\[\[[^]]+\]]', sentence)
        for s in sentences:
            sentence = sentence.replace(s, s[1:-1])
        for predicate in getPredicates(sentence):
            attributes = getAttributes(predicate)
            if ''.join(attributes).islower():
                sentence = sentence.replace(match[1],SKOLEM_CONSTANTS.pop(0))
            else:
                aL = [a for a in attributes if a.islower()] #lowercase attribute
                aU = [a for a in attributes if not a.islower()][0] #uppercase attribute
                sentence = sentence.replace(aU, f'{SKOLEM_CONSTANTS.pop(0)}({aL[0] if len(aL) else match[1]})')
    return sentence

In [58]:
def FOL_To_CNF(fol):
    statement = fol
    #removing the biconditional
    while '⇔' in statement:
        i = statement.index('⇔')
        new_statement = '[' + statement[:i] + '⇒' + statement[i+1:] + '] ∧ ['+ statement[i+1:] + '⇒' + statement[:i] + ']'
        statement = new_statement
    expr = '\[([^]]+)\]'
    statements = re.findall(expr, statement)
    
    #dividing the statement to statemnts based on [ ]
    for i, s in enumerate(statements):
        if '[' in s and ']' not in s:
            statements[i] += ']'
    for s in statements:
        statement = statement.replace(s, FOL_To_CNF(s))
    
    #removing the conditional
    while '⇒' in statement:
        i = statement.index('⇒')
        br = statement.index('[') if '[' in statement else 0
        new_statement = '¬' + statement[br:i] + '∨' + statement[i+1:]
        statement = statement[:br] + new_statement if br > 0 else new_statement
    
    #moving the negation 
    while '¬∀' in statement:
        i = statement.index('¬∀')
        statement = list(statement)
        statement[i], statement[i+1], statement[i+2] = '∃', statement[i+2], '¬'
        statement = ''.join(statement)
    while '¬∃' in statement:
        i = statement.index('¬∃')
        s = list(statement)
        s[i], s[i+1], s[i+2] = '∀', s[i+2], '¬'
        statement = ''.join(s)
    statement = statement.replace('¬[∀','[¬∀')
    statement = statement.replace('¬[∃','[¬∃')
    
    #rechecking the statements and applying the demorgans to covert if necessary
    expr = '(¬[∀∨∃].)'
    statements = re.findall(expr, statement)
    for s in statements:
        statement = statement.replace(s, FOL_To_CNF(s))
    expr = '¬\[[^]]+\]'
    statements = re.findall(expr, statement)
    for s in statements:
        statement = statement.replace(s, deMorConv(s))
    return statement

In [60]:
sentence_in_FOL = input("Enter sentence in First Order Logic: ")
print("FOL converted to CNF: {} \n\n".format(Skolemization(FOL_To_CNF(sentence_in_FOL))))

Enter sentence in First Order Logic: ∃x Teacher(x)∧Honest(x)
FOL converted to CNF:  Teacher(A)∧Honest(A) 




In [ ]:
#input testing
¬∀x Father(x) ⇒ Parent(x)
∀x Father(x) ⇔ Parent(x)
∃x Father(x) ⇒ Parent(x)

∀x Student(x) ∨ Smart(x) ⇒ GoodCitizen(x)